In [ ]:
import os

# Temporarily set the Hugging Face token (only for this runtime session)
os.environ["HUGGINGFACE_TOKEN"] = ""


In [2]:
! pip install -U "huggingface_hub[cli]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.33.2
    Uninstalling huggingface-hub-0.33.2:
      Successfully uninstalled huggingface-hub-0.33.2


In [3]:

import os
hf_token = os.getenv("HUGGINGFACE_TOKEN")
if not hf_token:
    raise EnvironmentError("❌ HUGGINGFACE_TOKEN not found. Set it using os.environ['HUGGINGFACE_TOKEN'] = 'your_token_here'")


In [4]:

from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", use_auth_token=hf_token)
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", use_auth_token=hf_token)


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:934: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [5]:

paragraph = """Remote work improves employee productivity. People working remotely often report fewer distractions and better focus.
EVs have zero tailpipe emissions. Most EVs are charged with electricity generated from fossil fuels.
AI-generated images mimic style but lack original thought. 3D-printed organs could reduce transplant wait times."""


In [6]:

import re

def split_into_sentences(text):
    return re.split(r'(?<=[.!?])\s+', text.strip())

sentences = split_into_sentences(paragraph)


In [7]:

import torch

def run_prompt(prompt, max_new_tokens=10):
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer.eos_token_id
        )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result[len(prompt):].strip()


In [8]:

classified = []
claims = []

for s in sentences:
    prompt = (
        "Classify the following sentence as either a 'claim' or a 'premise':\n"
        f"\"{s}\"\nAnswer:"
    )
    response = run_prompt(prompt)
    label = "claim" if "claim" in response.lower() else "premise"
    classified.append({"text": s, "type": label})
    if label == "claim":
        claims.append(s)


In [9]:

structured_output = {
    "claims": [{"text": entry["text"]} for entry in classified if entry["type"] == "claim"],
    "premises": [{"text": entry["text"]} for entry in classified if entry["type"] == "premise"]
}

import json
print(json.dumps(structured_output, indent=2))


{
  "claims": [],
  "premises": [
    {
      "text": "Remote work improves employee productivity."
    },
    {
      "text": "People working remotely often report fewer distractions and better focus."
    },
    {
      "text": "EVs have zero tailpipe emissions."
    },
    {
      "text": "Most EVs are charged with electricity generated from fossil fuels."
    },
    {
      "text": "AI-generated images mimic style but lack original thought."
    },
    {
      "text": "3D-printed organs could reduce transplant wait times."
    }
  ]
}
